In [5]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("Research essay A0284714Y.pdf")

documents = loader.load()


Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 26 0 (offset 0)
Ignoring wrong pointing object 31 0 (offset 0)
Ignoring wrong pointing object 39 0 (offset 0)
Ignoring wrong pointing object 41 0 (offset 0)
Ignoring wrong pointing object 43 0 (offset 0)


In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

textSplitting = RecursiveCharacterTextSplitter(
    chunk_size = 600, chunk_overlap = 0, 
    length_function = len, is_separator_regex = False
)

chunks = textSplitting.split_documents(documents)

print(len(chunks))

33


In [8]:
from sentence_transformers import SentenceTransformer
import numpy

embedding_model = SentenceTransformer('Snowflake/snowflake-arctic-embed-l', trust_remote_code=True)

chunk_texts = [chunk.page_content for chunk in chunks]

embeddings = embedding_model.encode(chunk_texts, show_progress_bar=True)

print(embeddings)

embeddings_np = numpy.array(embeddings).astype('float32') #fass takes in 



Batches: 100%|██████████| 2/2 [00:08<00:00,  4.12s/it]

[[ 0.00747336 -0.08902401  0.02077459 ... -0.01880907 -0.04058758
  -0.00896567]
 [ 0.01578886 -0.03381769  0.01899362 ...  0.00736172  0.02683345
   0.00205374]
 [ 0.00219827 -0.05417675  0.01220736 ... -0.02254741 -0.00642097
  -0.01477654]
 ...
 [ 0.01143975 -0.08681266 -0.00466315 ... -0.01262976  0.0092701
  -0.02335935]
 [ 0.00517226 -0.0447894  -0.00024153 ... -0.0359154  -0.01128982
  -0.05130968]
 [-0.03160997 -0.07529054  0.00959112 ...  0.05392884 -0.01141342
   0.00206799]]


In [9]:
import faiss #facebook ai similarity search 

query = "tourism in india"

query_embedding = embedding_model.encode(query, show_progress_bar=True)

query_embedding_np = numpy.array([query_embedding]).astype('float32')

nearest_neighbours = 5

dimension = embeddings_np.shape[1]

index = faiss.IndexFlatL2(dimension)
 
index.add(embeddings_np)

distance, indices = index.search(query_embedding_np, nearest_neighbours)

#The lower the distance, the more similar the chunk is to your query.
#hese are the positions of the most similar chunks within your original chunks list

print("Distance: ", distance)


print("Indices: ", indices)

similarity_scores = [ 1 / (1 + dist) for dist in distance[0] ]

print("Similarity scores: ", similarity_scores)


Batches: 100%|██████████| 1/1 [00:01<00:00,  1.52s/it]

Distance:  [[0.48538676 0.49017376 0.49785894 0.5116794  0.52909684]]
Indices:  [[24 27 25 13  3]]
Similarity scores:  [0.6732253360360179, 0.6710626832954101, 0.6676196084003985, 0.661515922467878, 0.653980815770242]


In [ ]:
#Retrieving the chunks using distance and indices

for i in range(len(indices)) :
    for rank in range(nearest_neighbours):
        chunk_index = indices[i][rank]
        similarity = similarity_scores[rank]
        chunk_text = chunks[chunk_index]
        print(f"Rank {rank+1}: {chunk_text.page_content} with similarity {similarity}")
        

Rank 1: showcasing the diverse offerings of India create positive content and change the perception of people.   Furthermore, there is immense potential for further growth and collaboration in tourism promotion between Singapore and India. This could involve joint marketing campaigns, cultural exchange programs, and facilitation of business partnerships in the tourism sector. Policymakers, tourism agencies, and stakeholders should prioritize initiatives that leverage the strengths of both countries and address existing challenges.  Conclusion: with similarity 0.6732257817797441
Rank 2: India looks promising, with immense potential for joint marketing campaigns, cultural exchange programs, and facilitation of business partnerships in the sector. with similarity 0.6710628711857465
Rank 3: Tourism has played a pivotal role in strengthening the multifaceted relationship between India and Singapore. The thriving tourism industry has fostered political, economic, and cultural ties between th

In [ ]:
#LLM 
import os
import google.generativeai as genai

api_key = os.getenv("GENAI_API_KEY")
if not api_key:
    raise ValueError("API key not found in environment variables.")

genai.configure(api_key=api_key)

llm = genai.GenerativeModel('gemini-1.5-flash')


In [ ]:
prompt = " Please answer the following question in context of the provided text:\n"
prompt += "Question: " + query + "\n"
for i in range(len(indices)) :
    for rank in range(nearest_neighbours):
        chunk_index = indices[i][rank]
        similarity = similarity_scores[rank]
        chunk_text = chunks[chunk_index]
        prompt += "Context: " + chunk_text.page_content + "\n"
        
response = llm.generate_content(prompt)

print(response)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "The provided text focuses on the **positive impact of tourism on the relationship between Singapore and India**, rather than providing a detailed analysis of tourism in India itself.  However, we can glean some insights:\n\n* **India's tourism offerings are diverse and hold immense potential:** The text highlights that India's offerings are diverse and capable of changing perceptions. \n* **India is a popular destination for Singaporean tourists:** The text mentions that India's \"splendour\" draws \"a substantial number of Singaporean visitors each year.\"\n* **Collaboration in tourism promotion between Singapore and India:** There is significant potential for joint marketing campaigns, cultural exchange programs, and business partnerships to further boost 